# Evaluating classifiers using confusion matrices, ROC curves, and AUC

In [1]:
import pandas as pd

X = pd.read_csv('titanic.X.csv', index_col='PassengerId')
y = pd.read_csv('titanic.y.csv', index_col='PassengerId')

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y['Survived'], test_size=0.3)

In [23]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

classifiers = [
    LinearSVC(random_state=0),
    KNeighborsClassifier(),
    RandomForestClassifier(random_state=0),
    AdaBoostClassifier(random_state=0)
]

for clf in classifiers:
    clf.fit(X_train, y_train)
    